Neural network model (without day 0)
 

In [ ]:
#datasets without day 0 
import pandas as pd
Imputed_test_set = pd.read_csv("Data/Imputed_test_set.csv")
Imputed_train_set = pd.read_csv("Data/Imputed_train_set.csv")
Imputed_train_set_up = pd.read_csv("Data/Imputed_train_set_up.csv")
Imputed_validation_set = pd.read_csv("Data/Imputed_validation_set.csv")

#Defining x and y values

##Train set

In [ ]:
#Extracting x and y-values as model input. Sequential sensordata is stored in a 3d matrix of 21 days per 5 features per 365 cows.
import numpy as np
import pandas as pd
#extracting all unique combinations of cow and calving moment 
Unique_Calvings = Imputed_train_set[['AnimalEartag', 'PaperRecordedCalvingDate']].drop_duplicates() 

#define sensors
feature_names = ["WalkingTimeMinutesPerDay", "EatingTimeMinutesPerDay", "LyingTimeMinutesPerDay", "StandingTimeMinutesPerDay", "RuminationTimeMinutesPerDay"]

#create empty lists per feature to store variables
AnimalEartag_list = [] 
PaperRecordedCalvingDate_list = []
CalciumValue_list = []
Calciumcluster_list = []
BCS_lijst = []
Loco_lijst = []
SensorWaardes_list = np.zeros((365,21,5)) #define matrix size for sequential features
j = 0

#itterating through the dataset in order to extract x and y for each cow 
for index, (AnimalEartag, PaperRecordedCalvingDate) in Unique_Calvings.iterrows():
    filter1 = Imputed_train_set['AnimalEartag'] == AnimalEartag
    filter2 = PaperRecordedCalvingDate == Imputed_train_set['PaperRecordedCalvingDate']
    df_calving = Imputed_train_set[filter1 & filter2]
    Loco = df_calving['FirstLocomotionScore'].max()
    BCS =  df_calving['FirstBCSScore'].max()
    cacluster = df_calving['Calciumcluster'].iloc[-1] 
    ca = df_calving['Cut_Off'].iloc[-1]
    sw = df_calving[feature_names]
   
    
  
    #convert to numpy
    sw_numpy = np.array(sw)
    #add to list 
    AnimalEartag_list.append(AnimalEartag)
    PaperRecordedCalvingDate_list.append(PaperRecordedCalvingDate)
    CalciumValue_list.append(ca)
    Calciumcluster_list.append(cacluster)
    BCS_lijst.append((BCS))
    Loco_lijst.append((Loco))
    
   
    SensorWaardes_list[j] = sw_numpy
    j = j + 1
 
#convert to numpy arrays
x_train = np.array(SensorWaardes_list)
y_train = np.asarray(Calciumcluster_list) #based on cluster method 
y_train2 = np.asarray(CalciumValue_list) #alternative method of categorisation based on cut-off value
x_BCS = np.asarray(BCS_lijst)
x_Loco = np.asarray(Loco_lijst)
x_static = np.stack((x_BCS, x_Loco), axis = 1)

In [ ]:
#Calvingseason and parity are cathegorial variables, however the models functions on numerical variables only, therefore the variables are converted into binairy variables using the sklearn onehotencoder. Each category gets its own colummn and the column of the category that is true gets a one, all other get a zero.     
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
#extract value for each cow
static = Imputed_train_set.groupby(['AnimalEartag', 'PaperRecordedCalvingDate']).first()
#convert to numpy
season = static['CalvingSeason'].to_numpy()
Parity = static['Parity'].to_numpy()
CalciumDaysInMilk = static['CalciumDaysInMilk'].to_numpy()
#define labelencoder
labelEnc = LabelEncoder()
#fit and apply
x_enc = labelEnc.fit_transform(season)
x_enc = x_enc.reshape(len(x_enc), 1) 
onehotEnc = OneHotEncoder(sparse=False)
season_encoded = onehotEnc.fit_transform(x_enc)
x_enc_P = labelEnc.fit_transform(Parity)
x_enc_P = x_enc_P.reshape(len(x_enc_P), 1) 
parity_encoded = onehotEnc.fit_transform(x_enc_P)
#combine season, parity and day of measurement into one numpy array 
x_static_lean = np.column_stack([season_encoded, parity_encoded, CalciumDaysInMilk])

In [ ]:
#BCS and Locomotion score was not measured for every cow, therefore some cows have a null value. The null values are replaced by the value that is most frequent in that colummn. 
import numpy as np
from sklearn.impute import SimpleImputer
imp_freq = SimpleImputer(missing_values=0.0, strategy='most_frequent')
imp_freq.fit(x_static)
x_static = imp_freq.transform(x_static)
#all static features are added up into one numpy array 
x_Static = np.column_stack([x_static, season_encoded, parity_encoded, CalciumDaysInMilk]) 

##Train set upsampeld

In [ ]:
#Since this train set is upsampled, it is impossible to filter on animal eartag and paperrecordedcalvingdate (some cows of cathegory 1 are duplicated and therefore no longer unique) therefore a new column was introduced while performing upsampling called 'Samplenumber' In order to extract an x and an y per sample, an itteration is performed based on samplenumber. 
Unique_Calvings = Imputed_train_set_up['SampleNumber'].drop_duplicates()


#creating empty lists, pay attention to the increased matrix size and the addition of a list for samplenumber
SampleNumber_list = []
CalciumValue_list = []
Calciumcluster_list = []
SensorWaardes_list = np.zeros((534,21,5)) #534 cows, 21 days, 5 sensors
BCS_lijst = []
Loco_lijst = []
j = 0
for s in range(len(Unique_Calvings)):
    filter = Imputed_train_set_up['SampleNumber'] == s
    df_calving = Imputed_train_set_up[filter]
    Loco = df_calving['FirstLocomotionScore'].max()
    BCS =  df_calving['FirstBCSScore'].max()
    i = 1
    cacluster = df_calving['Calciumcluster'].iloc[-i]
    ca = df_calving['Cut_Off'].iloc[-i]
    i = i + 1 #prevents infinite loop
  
    sw = df_calving[feature_names]
  
    #convert to numpy
    sw_numpy = np.array(sw)
    #add to list
    SampleNumber_list.append(s)
    CalciumValue_list.append(ca)
    Calciumcluster_list.append(cacluster)
    BCS_lijst.append((BCS))
    Loco_lijst.append((Loco))
  
    SensorWaardes_list[j] = sw_numpy
    j = j + 1

#transform to numpy array
x_train_up = np.array(SensorWaardes_list)
y_train_up = np.asarray(Calciumcluster_list)
y_train2_up =np.asarray(CalciumValue_list)
x_BCS_up = np.asarray(BCS_lijst) #BCS end dry period 
x_Loco_up = np.asarray(Loco_lijst) #locomotionscore end dry period
x_static_up = np.stack((x_BCS_up, x_Loco_up), axis = 1) #combining BCS and Locomotion in order to be able to perform upsampling efficiently

In [ ]:
#one hot encoding upsampled train set
#Calvingseason and parity are cathegorial variables, however the models functions on numerical variables only, therefore the variables are converted into binairy variables using the sklearn onehotencoder. Each category gets its own colummn and the column of the category that is true gets a one, all other get a zero.
#based on samplenumber instead of animaleartag due to the upsampling 
static = Imputed_train_set_up.groupby(['SampleNumber']).first()
#convert to numpy arrays
season = static['CalvingSeason'].to_numpy()
Parity = static['Parity'].to_numpy()
CalciumDaysInMilk_up = static['CalciumDaysInMilk'].to_numpy()
#define encoder
labelEnc = LabelEncoder()
#fit and apply
x_enc = labelEnc.fit_transform(season)
x_enc = x_enc.reshape(len(x_enc), 1) 
onehotEnc = OneHotEncoder(sparse=False)
season_encoded_up = onehotEnc.fit_transform(x_enc)
x_enc_P = labelEnc.fit_transform(Parity)
x_enc_P = x_enc_P.reshape(len(x_enc_P), 1) 
parity_encoded_up = onehotEnc.fit_transform(x_enc_P)
x_static_up = imp_freq.transform(x_static_up)
#combine parity, calving season and day of measurement
x_static_up_lean = np.column_stack([season_encoded_up, parity_encoded_up, CalciumDaysInMilk_up])
#combine all static features into one array
x_Static_up = np.column_stack([x_static_up, season_encoded_up, parity_encoded_up, CalciumDaysInMilk_up]) 


##Validation set

In [ ]:
#extracting all unique combinations of cow and calving moment  
Unique_Calvings_Val = Imputed_validation_set[['AnimalEartag', 'PaperRecordedCalvingDate']].drop_duplicates()


AnimalEartagV_list = []
PaperRecordedCalvingDateV_list = []
CalciumValueV_list = []
CalciumclusterV_list = []
BCS_lijst = []
Loco_lijst = []
SensorWaardesV_list = np.zeros((122,21,5)) #122 cows, 21 days, 5 sensors
j = 0
for index, (AnimalEartag, PaperRecordedCalvingDate) in Unique_Calvings_Val.iterrows():
    filter1 = Imputed_validation_set['AnimalEartag'] == AnimalEartag
    filter2 = PaperRecordedCalvingDate == Imputed_validation_set['PaperRecordedCalvingDate']
    df_calving_val = Imputed_validation_set[filter1 & filter2]
    i = 1
    Loco = df_calving_val['FirstLocomotionScore'].max()
    BCS =  df_calving_val['FirstBCSScore'].max()
    caV = df_calving_val['Cut_Off'].iloc[-i]
    caclusterV = df_calving_val['Calciumcluster'].iloc[-i]
    swV = df_calving_val[feature_names]
  
    #convert to numpy
    swV_numpy = np.array(swV)
    #add to list 
    AnimalEartagV_list.append(AnimalEartag)
    PaperRecordedCalvingDateV_list.append(PaperRecordedCalvingDate)
    CalciumValueV_list.append(caV)
    CalciumclusterV_list.append(caclusterV)
    BCS_lijst.append((BCS))
    Loco_lijst.append((Loco))
    
    SensorWaardesV_list[j] = swV_numpy
    j = j + 1

#convert to numpy 
x_val = np.array(SensorWaardesV_list)
y_val2 = np.asarray(CalciumValueV_list)
y_val = np.asarray(CalciumclusterV_list)
x_BCS = np.asarray(BCS_lijst)
x_Loco = np.asarray(Loco_lijst)
#stack BCS en locomotion score 
x_static_val = np.stack((x_BCS, x_Loco), axis = 1)

In [ ]:
#one hot encoding validation
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
static = Imputed_validation_set.groupby(['AnimalEartag', 'PaperRecordedCalvingDate']).first()
season = static['CalvingSeason'].to_numpy()
Parity = static['Parity'].to_numpy()
CalciumDaysInMilk = static['CalciumDaysInMilk'].to_numpy()
labelEnc = LabelEncoder()
x_enc = labelEnc.fit_transform(season)
x_enc = x_enc.reshape(len(x_enc), 1) 
onehotEnc = OneHotEncoder(sparse=False)
season_encoded = onehotEnc.fit_transform(x_enc)
x_enc_P = labelEnc.fit_transform(Parity)
x_enc_P = x_enc_P.reshape(len(x_enc_P), 1) 
parity_encoded = onehotEnc.fit_transform(x_enc_P)
#combine static features 
x_static_val_lean = np.column_stack([season_encoded, parity_encoded, CalciumDaysInMilk])
#impute missing BCS and Locomotion scores 
x_static_val = imp_freq.transform(x_static_val)
# combine BCS and Locomotion scores with the other static features
x_Static_val = np.column_stack([x_static_val, season_encoded, parity_encoded,CalciumDaysInMilk]) 


##Test set

In [ ]:
#extracting all unique combinations of cow and calving moment
Unique_Calvings_Test = Imputed_test_set[['AnimalEartag', 'PaperRecordedCalvingDate']].drop_duplicates()

AnimalEartag_Test_list = []
PaperRecordedCalvingDate_Test_list = []
CalciumValue_Test_list = []
Calciumcluster_Test_list = []
BCS_lijst = []
Loco_lijst = []
SensorWaardes_Test_list = np.zeros((122,21,5))
k = 0
for index, (AnimalEartag, PaperRecordedCalvingDate) in Unique_Calvings_Test.iterrows():
    filter1 = Imputed_test_set['AnimalEartag'] == AnimalEartag
    filter2 = PaperRecordedCalvingDate == Imputed_test_set['PaperRecordedCalvingDate']
    df_calving_test = Imputed_test_set[filter1 & filter2]
    Loco = df_calving_test['FirstLocomotionScore'].max()
    BCS =  df_calving_test['FirstBCSScore'].max()
    i = 1
    catest = df_calving_test['Cut_Off'].iloc[-i]
    caclustertest = df_calving_test['Calciumcluster'].iloc[-i]
    a = df_calving_test['CalvingSeason'].dropna()
  
    swtest = df_calving_test[feature_names]

    #convert to numpy
    swtest_numpy = np.array(swtest)

    #add to list 
    AnimalEartag_Test_list.append(AnimalEartag)
    PaperRecordedCalvingDate_Test_list.append(PaperRecordedCalvingDate)
    CalciumValue_Test_list.append(catest)
    Calciumcluster_Test_list.append(caclustertest)
    BCS_lijst.append((BCS))
    Loco_lijst.append((Loco))
    SensorWaardes_Test_list[k] = swtest_numpy
    k = k + 1


x_test = np.asarray(SensorWaardes_Test_list)
y_test2 = np.asarray(CalciumValue_Test_list)
y_test = np.asarray(Calciumcluster_Test_list)
x_BCS = np.asarray(BCS_lijst)
x_Loco = np.asarray(Loco_lijst)
x_static_test = np.stack((x_BCS, x_Loco), axis = 1)
#impute missing BCS and Locomotion scores
x_static_test = imp_freq.transform(x_static_test)

In [ ]:
#one hot encoding test set
static = Imputed_test_set.groupby(['AnimalEartag', 'PaperRecordedCalvingDate']).first()
season = static['CalvingSeason'].to_numpy()
Parity = static['Parity'].to_numpy()
CalciumDaysInMilk = static['CalciumDaysInMilk'].to_numpy()
labelEnc = LabelEncoder()
x_enc = labelEnc.fit_transform(season)
x_enc = x_enc.reshape(len(x_enc), 1) 
onehotEnc = OneHotEncoder(sparse=False)
season_encoded = onehotEnc.fit_transform(x_enc)
x_enc_P = labelEnc.fit_transform(Parity)
x_enc_P = x_enc_P.reshape(len(x_enc_P), 1) 
parity_encoded = onehotEnc.fit_transform(x_enc_P)
#combine static features
x_static_test_lean = np.column_stack([season_encoded, parity_encoded, CalciumDaysInMilk])
# combine static features with BCS and Locomotion scores
x_Static_test = np.column_stack([x_static_test, season_encoded, parity_encoded, CalciumDaysInMilk]) 


#Bootstrapping

In [ ]:
#XgBoost and Deep learning models have a form of randomness in their initialisation. The exact same model structure with the same training data may therefore give different results each time it is run. In order to be able to compare different models, model performance is not objective enough since it can differ with the same model and could be a high value simply because you were lucky. To compansate for this behaviour a second metric is proposed to compare models; the variance of the model. When a model has high variance, the chances are higher that a high value is obtained by luck and it could be that the second time the model is run, the exact same model results in dramaticly low performance metrics. To be able to measure varriance bootstraps are built, bootstraps are samples from the validation set with the same size as the validation set but acquired with sampling with replacement. Therefore the same model can be tested on multiple validation sets and the results can be compared and the SD calculated. 

#define function for creating bootstraps
def create_bootstrap(x_sensor,x_static,y1, y2):
    #initialise empty list for bootstraps
    bootstrap_x_sensor = []
    bootstrap_x_static = []
    bootstrap_y1 = []
    bootstrap_y2 = []
    
    #required length of bootstrap 
    len_val = x_val.shape[0]
    
    #get random observation 
    for i in range(len_val):
        # get random index
        random_idx = np.random.choice(range(len_val), 1)
		# get random observation
        random_x_sensor = x_sensor[random_idx]
        random_x_static = x_static[random_idx]
        random_y1 = y1[random_idx]
        random_y2 = y2[random_idx]
        
		# add random observation to bootstrap
        bootstrap_x_sensor.append(random_x_sensor)
        bootstrap_x_static.append(random_x_static)
        bootstrap_y1.append(random_y1)
        bootstrap_y2.append(random_y2)
        
	# convert to numpy
    bootstrap_x_sensor = np.asarray(bootstrap_x_sensor) 
    bootstrap_x_static = np.asarray(bootstrap_x_static)
    bootstrap_y1 = np.asarray(bootstrap_y1)
    bootstrap_y2 = np.asarray(bootstrap_y2)

	# return	
    return(bootstrap_x_sensor, bootstrap_x_static, bootstrap_y1, bootstrap_y2)


In [ ]:
# define function to create bootstraps
def create_bootstraps(x_sensor,x_static,y1, y2, number_bootstraps):
	
# initialize bootstrap containers
    bootstrap_container_x_sensor = []
    bootstrap_container_x_static = []
    bootstrap_container_y1 = []
    bootstrap_container_y2 = []
		
	# create n bootstrap
    for i in range(number_bootstraps):
		# get bootstrap
        bootstrap_x_sensor, bootstrap_x_static, bootstrap_y1, bootstrap_y2 = create_bootstrap(x_sensor,x_static,y1, y2)
		# add to container
        bootstrap_container_x_sensor.append(bootstrap_x_sensor)
        bootstrap_container_x_static.append(bootstrap_x_static)
        bootstrap_container_y1.append(bootstrap_y1)
        bootstrap_container_y2.append(bootstrap_y2)

	# return
    return(bootstrap_container_x_sensor, bootstrap_container_x_static, bootstrap_container_y1, bootstrap_container_y2)

In [ ]:
#define function to evaluate model 
def evaluate_model(model, bootstrap_container_x_sensor, bootstrap_container_x_static, bootstrap_container_y1, bootstrap_container_y2):

	# initialize evaluation container
    performance_container = []

	# loop through bootstraps
    for i in range(len(bootstrap_container_x_sensor)):

		# get X
        bootstrap_x_sensor = bootstrap_container_x_sensor[i]
        bootstrap_x_static = bootstrap_container_x_static[i]
		# get y
        bootstrap_y1 = bootstrap_container_y1[i]
        bootstrap_y2 = bootstrap_container_y2[i]
        #reshape x from 3d to 2d for machine learning and from 4d to 3d for deep learning 
        bootstrap_x_sensor = bootstrap_x_sensor.reshape((bootstrap_x_sensor.shape[0], (bootstrap_x_sensor.shape[1]*bootstrap_x_sensor.shape[2]), bootstrap_x_sensor.shape[3])) 
        bootstrap_x_static = bootstrap_x_static.reshape(bootstrap_x_static.shape[0], (bootstrap_x_static.shape[1]*bootstrap_x_static.shape[2]))
        #for XgBoost convert to d-matrix, comment out when not using XgBoost!
#         bootstrap_x_sensor =  xgb.DMatrix(bootstrap_x_sensor)
		# get predictions #depending on model to evaluate, for some models static features need to be added, for log reg model do not use predict but predict_proba  
        preds = model.predict([bootstrap_x_sensor, bootstrap_x_static], verbose = 0)
        # preds = model.predict(bootstrap_x_sensor, verbose = 0)
		# get metric # first choose witch y set to test 1 = clustered, 2 = cut-off
        auc = roc_auc_score(bootstrap_y2, preds)
        
		# add to container
        performance_container.append(auc)
      
	# return
    return(performance_container)


#Neural network model

In [ ]:
#packages needed for model building, while using databricks pay attention to the fact that you need a machine learning cluster instead of the deafault cluster
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, concatenate, BatchNormalization
from matplotlib import pyplot as plt
from tensorflow import keras
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report


## Sequential model

In [ ]:
class_weights = {0:1, 1:1} #used to deal with classinbalance 

In [ ]:
#define earlystopping callback
my_callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights = True,
    )

In [ ]:
#basic sequential model 
#define model
model = Sequential()
model.add(LSTM(90, activation = 'relu', input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=False)) #left relu activation out since tanh en sigmoid already cause non linearity
model.add(BatchNormalization())
model.add(Dropout(0.4)) 
model.add(Dense(1, activation = "sigmoid")) #Sigmoid activation causes binary result 
#compile model
model.compile(optimizer = 'Adam', loss='binary_crossentropy', metrics = ['AUC', 'Precision', 'Recall']) 
model.summary()

In [ ]:
# fit model, added earlystopping and class weights
history = model.fit(x_train_up, y_train2_up, epochs=100, batch_size = 12, validation_data=(x_val, y_val2), class_weight = {0:1, 1:2}, callbacks = [my_callbacks]) 

#plot loss and val_loss for every epoch 
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

In [ ]:
#Bootstrapping 
#create 50 bootstraps for deep learning 
Bootstrap_x_sensor, Bootstrap_x_static, Bootstrap_y1, Bootstrap_y2  = create_bootstraps(x_val, x_static_val_lean, y_val, y_val2, 50) 
Bootstrap_performance = evaluate_model(model, Bootstrap_x_sensor, Bootstrap_x_static, Bootstrap_y1, Bootstrap_y2)

In [ ]:
#results bootstrapping, mean auc and std 
Mean_performance = np.mean(Bootstrap_performance)
SD_performance = np.std(Bootstrap_performance)
# df = pd.DataFrame(Bootstrap_performance, columns = ['AUC'])
# display(df[df["AUC"] < 0.5])
Mean_performance, SD_performance, my_callbacks.stopped_epoch

In [ ]:
import scipy.stats as st
# create 95% confidence interval
st.t.interval(confidence=0.95, df = len(Bootstrap_performance), loc = Mean_performance, scale = st.sem(Bootstrap_performance))

In [ ]:
#predict values test set 
pred = model.predict(x_test)
pred_clases = np.where(np.squeeze(pred) < 0.56, 0, 1) 

In [ ]:
#visualize model performance
CM = confusion_matrix(y_test2, pred_clases)
disp = ConfusionMatrixDisplay(confusion_matrix=CM)
disp.plot()
print(classification_report(y_test2, pred_clases))

In [ ]:
#evaluation metrics
sensitiviteit = CM[1,1]/(CM[1,0]+CM[1,1])
specificiteit = CM[0,0]/(CM[0,0]+CM[0,1])
ppv = CM[1,1]/(CM[1,1]+CM[0,1])
auc = roc_auc_score(y_test2, pred)
average_precision = average_precision_score(y_test2, pred)
print('sensitiviteit =', sensitiviteit, 'specificiteit =', specificiteit, 'ppv =', ppv, 'AUC =', auc, 'average_precision = ', average_precision)

In [ ]:
#precission recall plot
from sklearn.metrics import precision_recall_curve, average_precision_score, auc, plot_precision_recall_curve
from matplotlib import pyplot as plt
#calculate precision and recall
precision, recall, thresholds = precision_recall_curve(y_test, pred_clases)

#create precision recall curve
fig, ax = plt.subplots()
ax.plot(recall, precision, color='purple')

#add axis labels to plot
ax.set_title('Precision-Recall Curve')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')

#display plot
plt.show()

### Hyperparameter tuning sequential model

In [ ]:
#define class weight options
cw1 = {0:1, 1:1}  
cw2 = {0:1, 1:2}
cw3 = {0:1, 1:3}
cw4 = {0:1, 1:4}

In [ ]:
#def parameters to tune
p_dropout = [0, 0.1, 0.2, 0.3, 0.4]
p_units = [10, 20, 30, 40, 50, 60, 70, 80,90,100]
p_BN = [1,0]
p_BS = [12, 22, 32, 42]
p_cw = [cw1, cw2, cw3, cw4]

In [ ]:
# A functional model was used for hyperparameter tuning, works at the same way as a sequential model but is compatinble with multi-input and therefore needed when adding static features.  
import random
#define inputs, in this case single input model
sensor_input = keras.Input(shape=(21,5))
#create empty list
Results = []
#run multiple models but with a different random configuration of parameters each time
for i in range(200):
    units = random.choice(p_units)
    BN = random.choice(p_BN)
    dropout = random.choice(p_dropout)
    BS = random.choice(p_BS)
    Class_weights = random.choice(p_cw)
#define LSTM:

    LSTM_output = LSTM(units)(sensor_input) #relu toevoegen? 

    if BN == 1:
        BN_output = BatchNormalization()(LSTM_output)
    else: 
        BN_output = LSTM_output

#dropout layer
    Dropout_output = Dropout(dropout)(BN_output)

#define final layer
    prediction =Dense(1, activation = 'sigmoid')(Dropout_output)

#define model
    Model = keras.Model(
    inputs = sensor_input,
    outputs =prediction)
#compile model
    Model.compile(optimizer = 'Adam', loss='binary_crossentropy',  metrics = ['AUC', 'Precision', 'Recall']) 
#fit model, verbose set to zero to speed up the process, verbose = 1 shows learning process with intermediate AUC scores of train set 
    Model.fit(x_train, y_train, batch_size = BS, epochs=100, class_weight = Class_weights, validation_data=(x_val, y_val), callbacks = [my_callbacks],  use_multiprocessing = True,  verbose = 0 )
#predict values validation set 
    pred = Model.predict(x_val, verbose = 0)
    pred_clases = np.where(np.squeeze(pred) < 0.5, 0, 1) 
#add predictions to list and the associated hyperparameters    
    Results.append({
        'f1_score': f1_score(y_val, pred_clases),
        'AUC': roc_auc_score(y_val, pred), 
        'Average Precision': average_precision_score(y_val, pred), 
        'p_units': units,
        'p_dropout': dropout,
        'p_BN' : BN,
        'p_BS' : BS,
        'p_cw' : Class_weights
        })
#convert to DF
Results = pd.DataFrame(Results)
#sort modelresults
Results = Results.sort_values(by='AUC', ascending=False)
display(Results)

In [ ]:
# A functional model was used for hyperparameter tuning, works at the same way as a sequential model but is compatinble with multi-input and therefore needed when adding static features.  
import random
#define inputs, in this case single input model
sensor_input = keras.Input(shape=(21,5))
#create empty list
Results = []
#run multiple models but with a different random configuration of parameters each time
for i in range(200):
    units = random.choice(p_units)
    BN = random.choice(p_BN)
    dropout = random.choice(p_dropout)
    BS = random.choice(p_BS)
    Class_weights = random.choice(p_cw)
#define LSTM:

    LSTM_output = LSTM(units)(sensor_input) #relu toevoegen? 

    if BN == 1:
        BN_output = BatchNormalization()(LSTM_output)
    else: 
        BN_output = LSTM_output

#dropout layer
    Dropout_output = Dropout(dropout)(BN_output)

#define final layer
    prediction =Dense(1, activation = 'sigmoid')(Dropout_output)

#define model
    Model = keras.Model(
    inputs = sensor_input,
    outputs =prediction)
#compile model
    Model.compile(optimizer = 'Adam', loss='binary_crossentropy',  metrics = ['AUC', 'Precision', 'Recall']) 
#fit model, verbose set to zero to speed up the process, verbose = 1 shows learning process with intermediate AUC scores of train set 
    Model.fit(x_train, y_train2, batch_size = BS, epochs=100, class_weight = Class_weights, validation_data=(x_val, y_val2), callbacks = [my_callbacks],  use_multiprocessing = True,  verbose = 0 )
#predict values validation set 
    pred = Model.predict(x_val, verbose = 0)
    pred_clases = np.where(np.squeeze(pred) < 0.5, 0, 1) 
#add predictions to list and the associated hyperparameters    
    Results.append({
        'f1_score': f1_score(y_val2, pred_clases),
        'AUC': roc_auc_score(y_val2, pred), 
        'Average Precision': average_precision_score(y_val2, pred), 
        'p_units': units,
        'p_dropout': dropout,
        'p_BN' : BN,
        'p_BS' : BS,
        'p_cw' : Class_weights
        })
#convert to DF
Results = pd.DataFrame(Results)
#sort modelresults
Results = Results.sort_values(by='AUC', ascending=False)
display(Results)

In [ ]:
# A functional model was used for hyperparameter tuning, works at the same way as a sequential model but is compatinble with multi-input and therefore needed when adding static features.  
import random
#define inputs, in this case single input model
sensor_input = keras.Input(shape=(21,5))
#create empty list
Results = []
#run multiple models but with a different random configuration of parameters each time
for i in range(200):
    units = random.choice(p_units)
    BN = random.choice(p_BN)
    dropout = random.choice(p_dropout)
    BS = random.choice(p_BS)
    Class_weights = random.choice(p_cw)
#define LSTM:

    LSTM_output = LSTM(units)(sensor_input) #relu toevoegen? 

    if BN == 1:
        BN_output = BatchNormalization()(LSTM_output)
    else: 
        BN_output = LSTM_output

#dropout layer
    Dropout_output = Dropout(dropout)(BN_output)

#define final layer
    prediction =Dense(1, activation = 'sigmoid')(Dropout_output)

#define model
    Model = keras.Model(
    inputs = sensor_input,
    outputs =prediction)
#compile model
    Model.compile(optimizer = 'Adam', loss='binary_crossentropy',  metrics = ['AUC', 'Precision', 'Recall']) 
#fit model, verbose set to zero to speed up the process, verbose = 1 shows learning process with intermediate AUC scores of train set 
    Model.fit(x_train_up, y_train_up, batch_size = BS, epochs=100, class_weight = Class_weights, validation_data=(x_val, y_val), callbacks = [my_callbacks],  use_multiprocessing = True,  verbose = 0 )
#predict values validation set 
    pred = Model.predict(x_val, verbose = 0)
    pred_clases = np.where(np.squeeze(pred) < 0.5, 0, 1) 
#add predictions to list and the associated hyperparameters    
    Results.append({
        'f1_score': f1_score(y_val, pred_clases),
        'AUC': roc_auc_score(y_val, pred), 
        'Average Precision': average_precision_score(y_val, pred), 
        'p_units': units,
        'p_dropout': dropout,
        'p_BN' : BN,
        'p_BS' : BS,
        'p_cw' : Class_weights
        })
#convert to DF
Results = pd.DataFrame(Results)
#sort modelresults
Results = Results.sort_values(by='AUC', ascending=False)
display(Results)

In [ ]:
# A functional model was used for hyperparameter tuning, works at the same way as a sequential model but is compatinble with multi-input and therefore needed when adding static features.  
import random
#define inputs, in this case single input model
sensor_input = keras.Input(shape=(21,5))
#create empty list
Results = []
#run multiple models but with a different random configuration of parameters each time
for i in range(200):
    units = random.choice(p_units)
    BN = random.choice(p_BN)
    dropout = random.choice(p_dropout)
    BS = random.choice(p_BS)
    Class_weights = random.choice(p_cw)
#define LSTM:

    LSTM_output = LSTM(units)(sensor_input) #relu toevoegen? 

    if BN == 1:
        BN_output = BatchNormalization()(LSTM_output)
    else: 
        BN_output = LSTM_output

#dropout layer
    Dropout_output = Dropout(dropout)(BN_output)

#define final layer
    prediction =Dense(1, activation = 'sigmoid')(Dropout_output)

#define model
    Model = keras.Model(
    inputs = sensor_input,
    outputs =prediction)
#compile model
    Model.compile(optimizer = 'Adam', loss='binary_crossentropy',  metrics = ['AUC', 'Precision', 'Recall']) 
#fit model, verbose set to zero to speed up the process, verbose = 1 shows learning process with intermediate AUC scores of train set 
    Model.fit(x_train_up, y_train2_up, batch_size = BS, epochs=100, class_weight = Class_weights, validation_data=(x_val, y_val2), callbacks = [my_callbacks],  use_multiprocessing = True,  verbose = 0 )
#predict values validation set 
    pred = Model.predict(x_val, verbose = 0)
    pred_clases = np.where(np.squeeze(pred) < 0.5, 0, 1) 
#add predictions to list and the associated hyperparameters    
    Results.append({
        'f1_score': f1_score(y_val2, pred_clases),
        'AUC': roc_auc_score(y_val2, pred), 
        'Average Precision': average_precision_score(y_val2, pred), 
        'p_units': units,
        'p_dropout': dropout,
        'p_BN' : BN,
        'p_BS' : BS,
        'p_cw' : Class_weights
        })
#convert to DF
Results = pd.DataFrame(Results)
#sort modelresults
Results = Results.sort_values(by='AUC', ascending=False)
display(Results)

##Functional model

In [ ]:
#Multi-input functional model with both sequential as static input
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
#define inputs
sensor_input = keras.Input(shape=(21,5))
static_input = keras.Input(shape=(8))

#define LSTM 
LSTM_output = LSTM(100)(sensor_input)
#add batch normalisation 
BN_output = BatchNormalization()(LSTM_output)
#combine with static features
all_features = concatenate([BN_output, static_input])

#define MLP-layer  
MLP_output = Dense(60, activation = 'relu')(all_features)

#add dropout 
Dropout_output = Dropout(rate = 0.4)(MLP_output)

#define final layer 
prediction = Dense(1, activation = 'sigmoid')(Dropout_output)

#define model
Model = keras.Model(
    inputs = [sensor_input, static_input],
    outputs = prediction)
#compile model
Model.compile(optimizer = 'Adam', loss='binary_crossentropy',  metrics = ['AUC', 'Precision', 'Recall'])
Model.summary()

In [ ]:
#fit model, add class_weights and earlystopping as callback (my_callbacks has been previously defined)
history = Model.fit([x_train_up, x_static_up_lean], y_train2_up, batch_size = 12, epochs=100, class_weight = {0:1, 1:2}, validation_data=([x_val, x_static_val_lean], y_val2), callbacks = [my_callbacks], use_multiprocessing = True )
#plot loss and val_loss curve to keep track of model functioning
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

In [ ]:
#Bootstrapping 
#create 50 bootstraps for deep learning
Bootstrap_x_sensor, Bootstrap_x_static, Bootstrap_y1, Bootstrap_y2  = create_bootstraps(x_val, x_static_val_lean, y_val, y_val2, 50) 
Bootstrap_performance = evaluate_model(Model, Bootstrap_x_sensor, Bootstrap_x_static, Bootstrap_y1, Bootstrap_y2)

In [ ]:
#results bootstrapping, mean auc and std 
Mean_performance = np.mean(Bootstrap_performance)
SD_performance = np.std(Bootstrap_performance)
df = pd.DataFrame(Bootstrap_performance, columns = ['AUC'])
# display(df)
Mean_performance, SD_performance, my_callbacks.stopped_epoch

In [ ]:
#testing final model on the test set 
pred = Model.predict([x_test, x_static_test_lean])
pred_clases = np.where(np.squeeze(pred) < 0.5, 0, 1) 
CM = confusion_matrix(y_test2, pred_clases)
disp = ConfusionMatrixDisplay(confusion_matrix=CM)
disp.plot()
print(classification_report(y_test2, pred_clases))

In [ ]:
#evaluation metrics 
sensitiviteit = CM[1,1]/(CM[1,0]+CM[1,1])
specificiteit = CM[0,0]/(CM[0,0]+CM[0,1])
ppv = CM[1,1]/(CM[1,1]+CM[0,1])
auc = roc_auc_score(y_test2, pred) 

average_precision = average_precision_score(y_test2, pred)
print('sensitiviteit =', sensitiviteit, 'specificiteit =', specificiteit, 'ppv =', ppv, 'AUC =', auc, 'average_precision = ', average_precision)

In [ ]:
#cummalative accuracy, dependent of threshold chosen and
from sklearn.metrics import accuracy_score 
np.shape(y_test2)
# np.shape(pred)
y_test2 = y_test2.reshape(y_test2.shape[0], 1)
pred_clases = pred_clases.reshape(pred_clases.shape[0], 1) 
Pred = np.concatenate((pred, pred_clases,y_test2), axis =1)
PredPd = pd.DataFrame(Pred, columns= ['result', 'Prediction', 'True'])
Sorted = PredPd.sort_values(by=['result'], ascending=False)
#pick only the predictions that the model is most sure about
subselection = Sorted.iloc[0:30]
accuracy = subselection['True'].sum()/subselection['Prediction'].sum()*100
total = accuracy_score(y_test2, pred_clases)
sub = accuracy_score(subselection['True'], subselection['Prediction'])
print('accuracy =', accuracy,'%', 'total =', total, 'alternative =', sub)

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(y_test2,  pred)

#create ROC curve
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#make a precision recall plot 
from sklearn.metrics import precision_recall_curve, average_precision_score, auc, plot_precision_recall_curve
from matplotlib import pyplot as plt
#calculate precision and recall
precision, recall, thresholds = precision_recall_curve(y_test2, pred )

#create precision recall curve
fig, ax = plt.subplots()
ax.plot(recall, precision, color='purple')

#add axis labels to plot
ax.set_title('Precision-Recall Curve')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')

#display plot
plt.show()

###hyperparameter tuning functional model

In [ ]:
#define possible class-weights
cw1 = {0:1, 1:1} 
cw2 = {0:1, 1:2}
cw3 = {0:1, 1:3}
cw4 = {0:1, 1:3}

In [ ]:
#define possible hyperparameters
p_dropout = [0, 0.1, 0.2, 0.3, 0.4]
p_units = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
p_units2 = [10, 20, 30, 40, 50, 60, 70, 80]
p_BN = [0,1]
p_BS = [12, 22, 32, 42]
p_cw = [cw1, cw2, cw3, cw4]

In [ ]:
#hyperparameter tuning using random search for multi-input functional model
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
import random
#define inputs
sensor_input = keras.Input(shape=(21,5))
static_input = keras.Input(shape=(8))
#create empty list for modelresults
results = []
#define number of possible combination to try
for i in range(200):
    #choose random parameters
    units = random.choice(p_units)
    unitsMLP = random.choice(p_units2)
    BN = random.choice(p_BN)
    dropout = random.choice(p_dropout)
    BS = random.choice(p_BS)
    Class_weights = random.choice(p_cw)

    #define LSTM:

    LSTM_output = LSTM(units)(sensor_input)
    #add batchnormalisation depending on hyperparameter choice
    if BN == 1:
        BN_output = BatchNormalization()(LSTM_output)
    else: 
        BN_output = LSTM_output
    #combine with static features
    all_features = concatenate([BN_output, static_input])

    #define MLP layer 
    MLP_output = Dense(unitsMLP, activation = 'relu')(all_features)

    #add dropout
    Dropout_output = Dropout(dropout)(MLP_output)

    #define final layer
    prediction =Dense(1, activation = 'sigmoid')(Dropout_output)

    #define model
    Model = keras.Model(
    inputs = [sensor_input, static_input],
    outputs =prediction)
    #compile model
    Model.compile(optimizer = 'Adam', loss='binary_crossentropy',  metrics = ['AUC', 'Precision', 'Recall'])
    #fit model, add earlystopping and validation data 
    Model.fit([x_train, x_static_lean], y_train, batch_size = BS, epochs=100, class_weight = Class_weights, validation_data=([x_val,x_static_val_lean], y_val), callbacks = [my_callbacks],  use_multiprocessing = True )
    #predict y-values validation set
    pred = Model.predict([x_val, x_static_val_lean])
    pred_clases = np.where(np.squeeze(pred) < 0.5, 0, 1) 
    #append model results to list and the hyperparameters used to make the model 
    results.append({
        'f1_score': f1_score(y_val, pred_clases),
        'AUC': roc_auc_score(y_val, pred), 
        'Average Precision': average_precision_score(y_val, pred), 
        'p_units': units,
        'p_units2': unitsMLP,
        'p_dropout': dropout,
        'p_BN' : BN,
        'p_BS' : BS,
        'p_cw' : Class_weights               
        })
#convert list to dataframe
results = pd.DataFrame(results)
#sort by AUC
results = results.sort_values(by='AUC', ascending=False)
display(results)

In [ ]:
#hyperparameter tuning using random search for multi-input functional model
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
import random
#define inputs
sensor_input = keras.Input(shape=(21,5))
static_input = keras.Input(shape=(8))
#create empty list for modelresults
results = []
#define number of possible combination to try
for i in range(200):
    #choose random parameters
    units = random.choice(p_units)
    unitsMLP = random.choice(p_units2)
    BN = random.choice(p_BN)
    dropout = random.choice(p_dropout)
    BS = random.choice(p_BS)
    Class_weights = random.choice(p_cw)

    #define LSTM:

    LSTM_output = LSTM(units)(sensor_input)
    #add batchnormalisation depending on hyperparameter choice
    if BN == 1:
        BN_output = BatchNormalization()(LSTM_output)
    else: 
        BN_output = LSTM_output
    #combine with static features
    all_features = concatenate([BN_output, static_input])

    #define MLP layer 
    MLP_output = Dense(unitsMLP, activation = 'relu')(all_features)

    #add dropout
    Dropout_output = Dropout(dropout)(MLP_output)

    #define final layer
    prediction =Dense(1, activation = 'sigmoid')(Dropout_output)

    #define model
    Model = keras.Model(
    inputs = [sensor_input, static_input],
    outputs =prediction)
    #compile model
    Model.compile(optimizer = 'Adam', loss='binary_crossentropy',  metrics = ['AUC', 'Precision', 'Recall'])
    #fit model, add earlystopping and validation data 
    Model.fit([x_train_up, x_static_up_lean], y_train_up, batch_size = BS, epochs=100, class_weight = Class_weights, validation_data=([x_val,x_static_val_lean], y_val), callbacks = [my_callbacks],  use_multiprocessing = True )
    #predict y-values validation set
    pred = Model.predict([x_val, x_static_val_lean])
    pred_clases = np.where(np.squeeze(pred) < 0.5, 0, 1) 
    #append model results to list and the hyperparameters used to make the model 
    results.append({
        'f1_score': f1_score(y_val, pred_clases),
        'AUC': roc_auc_score(y_val, pred), 
        'Average Precision': average_precision_score(y_val, pred), 
        'p_units': units,
        'p_units2': unitsMLP,
        'p_dropout': dropout,
        'p_BN' : BN,
        'p_BS' : BS,
        'p_cw' : Class_weights               
        })
#convert list to dataframe
results = pd.DataFrame(results)
#sort by AUC
results = results.sort_values(by='AUC', ascending=False)
display(results)

In [ ]:
#hyperparameter tuning using random search for multi-input functional model
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
import random
#define inputs
sensor_input = keras.Input(shape=(21,5))
static_input = keras.Input(shape=(8))
#create empty list for modelresults
results = []
#define number of possible combination to try
for i in range(200):
    #choose random parameters
    units = random.choice(p_units)
    unitsMLP = random.choice(p_units2)
    BN = random.choice(p_BN)
    dropout = random.choice(p_dropout)
    BS = random.choice(p_BS)
    Class_weights = random.choice(p_cw)

    #define LSTM:

    LSTM_output = LSTM(units)(sensor_input)
    #add batchnormalisation depending on hyperparameter choice
    if BN == 1:
        BN_output = BatchNormalization()(LSTM_output)
    else: 
        BN_output = LSTM_output
    #combine with static features
    all_features = concatenate([BN_output, static_input])

    #define MLP layer 
    MLP_output = Dense(unitsMLP, activation = 'relu')(all_features)

    #add dropout
    Dropout_output = Dropout(dropout)(MLP_output)

    #define final layer
    prediction =Dense(1, activation = 'sigmoid')(Dropout_output)

    #define model
    Model = keras.Model(
    inputs = [sensor_input, static_input],
    outputs =prediction)
    #compile model
    Model.compile(optimizer = 'Adam', loss='binary_crossentropy',  metrics = ['AUC', 'Precision', 'Recall'])
    #fit model, add earlystopping and validation data 
    Model.fit([x_train, x_static_lean], y_train2, batch_size = BS, epochs=100, class_weight = Class_weights, validation_data=([x_val,x_static_val_lean], y_val2), callbacks = [my_callbacks],  use_multiprocessing = True, verbose = 0)
    #predict y-values validation set
    pred = Model.predict([x_val, x_static_val_lean])
    pred_clases = np.where(np.squeeze(pred) < 0.5, 0, 1) 
    #append model results to list and the hyperparameters used to make the model 
    results.append({
        'f1_score': f1_score(y_val2, pred_clases),
        'AUC': roc_auc_score(y_val2, pred), 
        'Average Precision': average_precision_score(y_val2, pred), 
        'p_units': units,
        'p_units2': unitsMLP,
        'p_dropout': dropout,
        'p_BN' : BN,
        'p_BS' : BS,
        'p_cw' : Class_weights               
        })
#convert list to dataframe
results = pd.DataFrame(results)
#sort by AUC
results = results.sort_values(by='AUC', ascending=False)
display(results)

In [ ]:
#hyperparameter tuning using random search for multi-input functional model
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
import random
#define inputs
sensor_input = keras.Input(shape=(21,5))
static_input = keras.Input(shape=(8))
#create empty list for modelresults
results = []
#define number of possible combination to try
for i in range(200):
    #choose random parameters
    units = random.choice(p_units)
    unitsMLP = random.choice(p_units2)
    BN = random.choice(p_BN)
    dropout = random.choice(p_dropout)
    BS = random.choice(p_BS)
    Class_weights = random.choice(p_cw)

    #define LSTM:

    LSTM_output = LSTM(units)(sensor_input)
    #add batchnormalisation depending on hyperparameter choice
    if BN == 1:
        BN_output = BatchNormalization()(LSTM_output)
    else: 
        BN_output = LSTM_output
    #combine with static features
    all_features = concatenate([BN_output, static_input])

    #define MLP layer 
    MLP_output = Dense(unitsMLP, activation = 'relu')(all_features)

    #add dropout
    Dropout_output = Dropout(dropout)(MLP_output)

    #define final layer
    prediction =Dense(1, activation = 'sigmoid')(Dropout_output)

    #define model
    Model = keras.Model(
    inputs = [sensor_input, static_input],
    outputs =prediction)
    #compile model
    Model.compile(optimizer = 'Adam', loss='binary_crossentropy',  metrics = ['AUC', 'Precision', 'Recall'])
    #fit model, add earlystopping and validation data 
    Model.fit([x_train_up, x_static_up_lean], y_train2_up, batch_size = BS, epochs=100, class_weight = Class_weights, validation_data=([x_val,x_static_val_lean], y_val2), callbacks = [my_callbacks],  use_multiprocessing = True, verbose = 0)
    #predict y-values validation set
    pred = Model.predict([x_val, x_static_val_lean])
    pred_clases = np.where(np.squeeze(pred) < 0.5, 0, 1) 
    #append model results to list and the hyperparameters used to make the model 
    results.append({
        'f1_score': f1_score(y_val2, pred_clases),
        'AUC': roc_auc_score(y_val2, pred), 
        'Average Precision': average_precision_score(y_val2, pred), 
        'p_units': units,
        'p_units2': unitsMLP,
        'p_dropout': dropout,
        'p_BN' : BN,
        'p_BS' : BS,
        'p_cw' : Class_weights               
        })
#convert list to dataframe
results = pd.DataFrame(results)
#sort by AUC
results = results.sort_values(by='AUC', ascending=False)
display(results)

In [ ]:
#hyperparameter tuning using random search for multi-input functional model
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
import random
#define inputs
sensor_input = keras.Input(shape=(21,5))
static_input = keras.Input(shape=(10))
#create empty list for modelresults
results = []
#define number of possible combination to try
for i in range(200):
    #choose random parameters
    units = random.choice(p_units)
    unitsMLP = random.choice(p_units2)
    BN = random.choice(p_BN)
    dropout = random.choice(p_dropout)
    BS = random.choice(p_BS)
    Class_weights = random.choice(p_cw)

    #define LSTM:

    LSTM_output = LSTM(units)(sensor_input)
    #add batchnormalisation depending on hyperparameter choice
    if BN == 1:
        BN_output = BatchNormalization()(LSTM_output)
    else: 
        BN_output = LSTM_output
    #combine with static features
    all_features = concatenate([BN_output, static_input])

    #define MLP layer 
    MLP_output = Dense(unitsMLP, activation = 'relu')(all_features)

    #add dropout
    Dropout_output = Dropout(dropout)(MLP_output)

    #define final layer
    prediction =Dense(1, activation = 'sigmoid')(Dropout_output)

    #define model
    Model = keras.Model(
    inputs = [sensor_input, static_input],
    outputs =prediction)
    #compile model
    Model.compile(optimizer = 'Adam', loss='binary_crossentropy',  metrics = ['AUC', 'Precision', 'Recall'])
    #fit model, add earlystopping and validation data 
    Model.fit([x_train, x_Static], y_train, batch_size = BS, epochs=100, class_weight = Class_weights, validation_data=([x_val,x_Static_val], y_val), callbacks = [my_callbacks],  use_multiprocessing = True, verbose = 0)
    #predict y-values validation set
    pred = Model.predict([x_val, x_Static_val])
    pred_clases = np.where(np.squeeze(pred) < 0.5, 0, 1) 
    #append model results to list and the hyperparameters used to make the model 
    results.append({
        'f1_score': f1_score(y_val, pred_clases),
        'AUC': roc_auc_score(y_val, pred), 
        'Average Precision': average_precision_score(y_val, pred), 
        'p_units': units,
        'p_units2': unitsMLP,
        'p_dropout': dropout,
        'p_BN' : BN,
        'p_BS' : BS,
        'p_cw' : Class_weights               
        })
#convert list to dataframe
results = pd.DataFrame(results)
#sort by AUC
results = results.sort_values(by='AUC', ascending=False)
display(results)

In [ ]:
#hyperparameter tuning using random search for multi-input functional model
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
import random
#define inputs
sensor_input = keras.Input(shape=(21,5))
static_input = keras.Input(shape=(10))
#create empty list for modelresults
results = []
#define number of possible combination to try
for i in range(200):
    #choose random parameters
    units = random.choice(p_units)
    unitsMLP = random.choice(p_units2)
    BN = random.choice(p_BN)
    dropout = random.choice(p_dropout)
    BS = random.choice(p_BS)
    Class_weights = random.choice(p_cw)

    #define LSTM:

    LSTM_output = LSTM(units)(sensor_input)
    #add batchnormalisation depending on hyperparameter choice
    if BN == 1:
        BN_output = BatchNormalization()(LSTM_output)
    else: 
        BN_output = LSTM_output
    #combine with static features
    all_features = concatenate([BN_output, static_input])

    #define MLP layer 
    MLP_output = Dense(unitsMLP, activation = 'relu')(all_features)

    #add dropout
    Dropout_output = Dropout(dropout)(MLP_output)

    #define final layer
    prediction =Dense(1, activation = 'sigmoid')(Dropout_output)

    #define model
    Model = keras.Model(
    inputs = [sensor_input, static_input],
    outputs =prediction)
    #compile model
    Model.compile(optimizer = 'Adam', loss='binary_crossentropy',  metrics = ['AUC', 'Precision', 'Recall'])
    #fit model, add earlystopping and validation data 
    Model.fit([x_train, x_Static], y_train2, batch_size = BS, epochs=100, class_weight = Class_weights, validation_data=([x_val,x_Static_val], y_val2), callbacks = [my_callbacks],  use_multiprocessing = True, verbose = 0)
    #predict y-values validation set
    pred = Model.predict([x_val, x_Static_val])
    pred_clases = np.where(np.squeeze(pred) < 0.5, 0, 1) 
    #append model results to list and the hyperparameters used to make the model 
    results.append({
        'f1_score': f1_score(y_val2, pred_clases),
        'AUC': roc_auc_score(y_val2, pred), 
        'Average Precision': average_precision_score(y_val2, pred), 
        'p_units': units,
        'p_units2': unitsMLP,
        'p_dropout': dropout,
        'p_BN' : BN,
        'p_BS' : BS,
        'p_cw' : Class_weights               
        })
#convert list to dataframe
results = pd.DataFrame(results)
#sort by AUC
results = results.sort_values(by='AUC', ascending=False)
display(results)

In [ ]:
#hyperparameter tuning using random search for multi-input functional model
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
import random
#define inputs
sensor_input = keras.Input(shape=(21,5))
static_input = keras.Input(shape=(10))
#create empty list for modelresults
results = []
#define number of possible combination to try
for i in range(200):
    #choose random parameters
    units = random.choice(p_units)
    unitsMLP = random.choice(p_units2)
    BN = random.choice(p_BN)
    dropout = random.choice(p_dropout)
    BS = random.choice(p_BS)
    Class_weights = random.choice(p_cw)

    #define LSTM:

    LSTM_output = LSTM(units)(sensor_input)
    #add batchnormalisation depending on hyperparameter choice
    if BN == 1:
        BN_output = BatchNormalization()(LSTM_output)
    else: 
        BN_output = LSTM_output
    #combine with static features
    all_features = concatenate([BN_output, static_input])

    #define MLP layer 
    MLP_output = Dense(unitsMLP, activation = 'relu')(all_features)

    #add dropout
    Dropout_output = Dropout(dropout)(MLP_output)

    #define final layer
    prediction =Dense(1, activation = 'sigmoid')(Dropout_output)

    #define model
    Model = keras.Model(
    inputs = [sensor_input, static_input],
    outputs =prediction)
    #compile model
    Model.compile(optimizer = 'Adam', loss='binary_crossentropy',  metrics = ['AUC', 'Precision', 'Recall'])
    #fit model, add earlystopping and validation data 
    Model.fit([x_train_up, x_Static_up], y_train_up, batch_size = BS, epochs=100, class_weight = Class_weights, validation_data=([x_val,x_Static_val], y_val), callbacks = [my_callbacks],  use_multiprocessing = True, verbose = 0)
    #predict y-values validation set
    pred = Model.predict([x_val, x_Static_val], verbose = 0)
    pred_clases = np.where(np.squeeze(pred) < 0.5, 0, 1) 
    #append model results to list and the hyperparameters used to make the model 
    results.append({
        'f1_score': f1_score(y_val, pred_clases),
        'AUC': roc_auc_score(y_val, pred), 
        'Average Precision': average_precision_score(y_val, pred), 
        'p_units': units,
        'p_units2': unitsMLP,
        'p_dropout': dropout,
        'p_BN' : BN,
        'p_BS' : BS,
        'p_cw' : Class_weights               
        })
#convert list to dataframe
results = pd.DataFrame(results)
#sort by AUC
results = results.sort_values(by='AUC', ascending=False)
display(results)

In [ ]:
#hyperparameter tuning using random search for multi-input functional model
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
import random
#define inputs
sensor_input = keras.Input(shape=(21,5))
static_input = keras.Input(shape=(10))
#create empty list for modelresults
results = []
#define number of possible combination to try
for i in range(200):
    #choose random parameters
    units = random.choice(p_units)
    unitsMLP = random.choice(p_units2)
    BN = random.choice(p_BN)
    dropout = random.choice(p_dropout)
    BS = random.choice(p_BS)
    Class_weights = random.choice(p_cw)

    #define LSTM:

    LSTM_output = LSTM(units)(sensor_input)
    #add batchnormalisation depending on hyperparameter choice
    if BN == 1:
        BN_output = BatchNormalization()(LSTM_output)
    else: 
        BN_output = LSTM_output
    #combine with static features
    all_features = concatenate([BN_output, static_input])

    #define MLP layer 
    MLP_output = Dense(unitsMLP, activation = 'relu')(all_features)

    #add dropout
    Dropout_output = Dropout(dropout)(MLP_output)

    #define final layer
    prediction =Dense(1, activation = 'sigmoid')(Dropout_output)

    #define model
    Model = keras.Model(
    inputs = [sensor_input, static_input],
    outputs =prediction)
    #compile model
    Model.compile(optimizer = 'Adam', loss='binary_crossentropy',  metrics = ['AUC', 'Precision', 'Recall'])
    #fit model, add earlystopping and validation data 
    Model.fit([x_train_up, x_Static_up], y_train2_up, batch_size = BS, epochs=100, class_weight = Class_weights, validation_data=([x_val,x_Static_val], y_val2), callbacks = [my_callbacks],  use_multiprocessing = True, verbose = 0)
    #predict y-values validation set
    pred = Model.predict([x_val, x_Static_val], verbose = 0)
    pred_clases = np.where(np.squeeze(pred) < 0.5, 0, 1) 
    #append model results to list and the hyperparameters used to make the model 
    results.append({
        'f1_score': f1_score(y_val2, pred_clases),
        'AUC': roc_auc_score(y_val2, pred), 
        'Average Precision': average_precision_score(y_val2, pred), 
        'p_units': units,
        'p_units2': unitsMLP,
        'p_dropout': dropout,
        'p_BN' : BN,
        'p_BS' : BS,
        'p_cw' : Class_weights               
        })
#convert list to dataframe
results = pd.DataFrame(results)
#sort by AUC
results = results.sort_values(by='AUC', ascending=False)
display(results)

Feature importance

In [ ]:
#test jaccard_score > supposed to be a good metric for performance for a binary classification mode, but first chose cut-off  
from sklearn.metrics import mean_squared_error, jaccard_score, accuracy_score
#make prediction
pred = Model.predict([x_val, x_static_val_lean])
#convert output to 0 or 1 
pred_clases = np.where(np.squeeze(pred) < 0.3, 0, 1)
#calculate performance metrics 
jaccard = jaccard_score(y_val2, pred_clases)
auc = roc_auc_score(y_val2, pred) 
accuracy = accuracy_score(y_val2, pred_clases)
print('jaccard =', jaccard, 'auc =', auc, 'accuracy =', accuracy)

In [ ]:
#same as previous cell but for sequential model 
#make prediction
pred = model.predict(x_val)
# convert to 0 or 1 
pred_clases = np.where(np.squeeze(pred) < 0.6, 0, 1)
#performce metrics 
jaccard = jaccard_score(y_val2, pred_clases)
auc = roc_auc_score(y_val2, pred) 
print('jaccard =', jaccard, 'auc =', auc)


Feature importance with static 

In [ ]:
from sklearn.utils import shuffle
from numpy import random
from sklearn.metrics import mean_squared_error, jaccard_score

sat_f_names = ["Autumn", "Spring", "Summer", "Winter", "Parity1", "Parity2", "Parity3+","CalciumDaysInMilk"]

#shuffle sensor features
def feature_importance(model, x_sensor, x_static, y, threshold, len_shuf):

   
    
    #required length of bootstrap 
    len_val = x_sensor.shape[2]
    
    #container
    performance_container = []
    
    #performance full (normal) model
    auc_full_mod = roc_auc_score(y,model.predict([x_sensor, x_static],verbose = 0))
    pred_clases_full = np.where(np.squeeze(model.predict([x_sensor, x_static],verbose = 0)) < threshold, 0, 1)
    jaccard_full_mod = jaccard_score(y,pred_clases_full)
    accuracy_full_mod = accuracy_score(y,pred_clases_full) 
    
    
    
    for i in range(len_val):
        for j in range(len_shuf):
            #set column to 0
            x_cop = x_sensor.copy()
            random.shuffle(x_cop[:,:,i])
            # x_cop[:,:,i] = 0
            #predict using alternative set 
            pred = model.predict([x_cop, x_static],verbose = 0)
            # pred classes based on threshold
            pred_clases = np.where(np.squeeze(pred) < threshold, 0, 1)
            #get performance metric
            auc = roc_auc_score(y, pred) 
            jaccard = jaccard_score(y,pred_clases)
            accuracy = accuracy_score(y,pred_clases)
            # MSE = mean_squared_error(y, pred, squared = False)
            #average_precision = average_precision_score(y, pred)
            
            # print("feat %i sim %i" % (i,j))
            #add to container 
            performance_container.append({"pred": feature_names[i], "delta_auc": auc_full_mod - auc, "delta_jaccard": jaccard_full_mod - jaccard, 'delta_accuracy': accuracy_full_mod - accuracy})
        
    return(performance_container)

#shuffle static features    
def feature_importance_static(model, x_sensor, x_static, y, threshold, len_shuf):

   
    
    #required length of bootstrap 
    len_val = x_static.shape[1]
    
    #container
    performance_container = []
    
    #performance full (normal) model
    auc_full_mod = roc_auc_score(y,model.predict([x_sensor, x_static],verbose = 0))
    pred_clases_full = np.where(np.squeeze(model.predict([x_sensor, x_static],verbose = 0)) < threshold, 0, 1)
    jaccard_full_mod = jaccard_score(y,pred_clases_full)
    accuracy_full_mod = accuracy_score(y,pred_clases_full) 
    
    
    for i in range(len_val):
        for j in range(len_shuf):
            #set column to 0
            
            x_cop = x_static.copy()
            random.shuffle(x_cop[:,i])
            # x_cop[:,:,i] = 0
            #predict using alternative set 
            pred = model.predict([x_sensor, x_cop],verbose = 0)
            #predict classes based on threshold
            pred_clases = np.where(np.squeeze(pred) < threshold, 0, 1)
            #get performance metric
            auc = roc_auc_score(y, pred) 
            jaccard = jaccard_score(y,pred_clases)
            accuracy = accuracy_score(y,pred_clases)
            # MSE = mean_squared_error(y, pred, squared = False)
            #average_precision = average_precision_score(y, pred)
            
            # print("feat %i sim %i" % (i,j))
            #add to container 
            performance_container.append({"pred": sat_f_names[i], "delta_auc": auc_full_mod - auc, "delta_jaccard": jaccard_full_mod - jaccard, 'delta_accuracy': accuracy_full_mod - accuracy})
        
    return(performance_container)


Feature importance sequantial models without static features

In [ ]:
from sklearn.utils import shuffle
from numpy import random
from sklearn.metrics import mean_squared_error, jaccard_score

def feature_importance_no_static(model, x_sensor, y, threshold, len_shuf):

   
    
    #required length of bootstrap 
    len_val = x_sensor.shape[2]
    
    #container
    performance_container = []
    
    #performance full (normal) model
    auc_full_mod = roc_auc_score(y,model.predict(x_sensor,verbose = 0))
    pred_clases_full = np.where(np.squeeze(model.predict(x_sensor,verbose = 0)) < threshold, 0, 1)
    jaccard_full_mod = jaccard_score(y,pred_clases_full)
    accuracy_full_mod = accuracy_score(y,pred_clases_full)
    
    
    
    for i in range(len_val):
        for j in range(len_shuf):
            #set column to 0
            x_cop = x_sensor.copy()
            random.shuffle(x_cop[:,:,i])
            # x_cop[:,:,i] = 0
            #predict using alternative set 
            pred = model.predict(x_cop, verbose = 0)
            # pred classes based on threshold
            pred_clases = np.where(np.squeeze(pred) < threshold, 0, 1)
            #get performance metric
            auc = roc_auc_score(y, pred) 
            jaccard = jaccard_score(y,pred_clases)
            accuracy = accuracy_score(y,pred_clases)
            #average_precision = average_precision_score(y, pred)
            
            # print("feat %i sim %i" % (i,j))
            #add to container 
            performance_container.append({"pred": feature_names[i], "delta_auc": auc_full_mod - auc, "delta_jaccard": jaccard_full_mod - jaccard, 'delta_accuracy': accuracy_full_mod - accuracy})
        
    return(performance_container)
    


In [ ]:
#feature importance for the model without static features 
result_feature_no_static = feature_importance_no_static(model, x_val, y_val2, 0.6, 50)
df_result_feature_no_static = pd.DataFrame(result_feature_no_static).groupby('pred').mean().sort_values(by = 'delta_accuracy', ascending = False)
df_result_feature_no_static 

In [ ]:
#feature importance per static feature 
result_perf_test_static = feature_importance_static(Model, x_val, x_static_val_lean, y_val2,0.3,50)
df_results_perf_stat = pd.DataFrame(result_perf_test_static).groupby('pred').mean().sort_values(by = 'delta_accuracy', ascending = False)
df_results_perf_stat 


In [ ]:
#feature importances per behaviour
result_perf_test = feature_importance(Model, x_val, x_static_val_lean, y_val2, 0.2, 50)
df_results_perf = pd.DataFrame(result_perf_test).groupby('pred').mean().sort_values(by = 'delta_accuracy',
                                              ascending = False)
df_results_perf 

In [ ]:
#Alternative feature importance whereby all values of a feature are converted to 0 and then performance metrics of the model are calculated
x_t_sens = np.zeros(x_val.shape)
#select columns no to convert to 0
x_t_sens[:,:,[11,17,14,18,16,8,5,10,12,3,0,7,2,13,6,1,15,4,9]] = x_val[:,:,[11,17,14,18,16,8,5,10,12,3,0,7,2,13,6,1,15,4,9]]
x_t_sens_static = np.zeros(x_static_val_lean.shape)
#predict
pred = Model.predict([x_val, x_t_sens_static])
auc = roc_auc_score(y_val2, pred) 
auc